In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
working_dir = "../input"
# Any results you write to the current directory are saved as output.

from sklearn.model_selection import KFold
from scipy.stats import ttest_ind_from_stats
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import f_regression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import  OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
import xgboost as xgb
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import os
from collections import Counter
from functools import cmp_to_key
import json

from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
import pickle as pkl


['.DS_Store', 'test', 'train_metadata', 'test_sentiment', 'train_sentiment', 'train', 'train_images']


Using TensorFlow backend.


> ## csv model

In [2]:
def load_csv(path):
    csv_dataset = pd.read_csv(path,index_col="PetID")
    return csv_dataset
csv_dataset = load_csv(working_dir+"/train/"+"train.csv")
print(csv_dataset.shape)
csv_dataset.head()

(14993, 23)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PhotoAmt,AdoptionSpeed
PetID,,,,,,,,,,,,,,,,,,,,,
86e1089a3,2,Nibble,3,299,0,1,1,7,0,1,...,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,2
6296e909a,2,No Name Yet,1,265,0,1,1,2,0,2,...,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,2.0,0
3422e4906,1,Brisco,1,307,0,1,2,7,0,2,...,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,7.0,3
5842f1ff5,1,Miko,4,307,0,2,1,2,0,2,...,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",8.0,2
850a43f90,1,Hunter,1,307,0,1,1,0,0,2,...,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,3.0,2


### we can see "Name" "Description" has NaN Value

In [3]:
csv_dataset.count()

Type             14993
Name             13736
Age              14993
Breed1           14993
Breed2           14993
Gender           14993
Color1           14993
Color2           14993
Color3           14993
MaturitySize     14993
FurLength        14993
Vaccinated       14993
Dewormed         14993
Sterilized       14993
Health           14993
Quantity         14993
Fee              14993
State            14993
RescuerID        14993
VideoAmt         14993
Description      14981
PhotoAmt         14993
AdoptionSpeed    14993
dtype: int64

In [4]:
print(csv_dataset["Name"].unique().shape)
count = Counter(csv_dataset["Name"].values.tolist())
sorted_count = sorted(count.items(), key = lambda x: x[1], reverse = True)
sorted_count

(9061,)


[(nan, 1257),
 ('Baby', 66),
 ('Lucky', 64),
 ('Brownie', 54),
 ('No Name', 54),
 ('Mimi', 52),
 ('Blackie', 49),
 ('Puppy', 45),
 ('Kitty', 39),
 ('Max', 39),
 ('Kittens', 39),
 ('Oreo', 36),
 ('Coco', 35),
 ('Tiger', 34),
 ('Angel', 33),
 ('Milo', 32),
 ('Bobby', 30),
 ('Snowy', 30),
 ('Lucy', 29),
 ('Kiki', 29),
 ('Lily', 28),
 ('Tom', 27),
 ('Ginger', 26),
 ('Puppies', 26),
 ('Prince', 25),
 ('Happy', 25),
 ('Daisy', 24),
 ('Kitten', 23),
 ('Simba', 23),
 ('Lola', 23),
 ('Bella', 23),
 ('Rocky', 23),
 ('No Name Yet', 22),
 ('Molly', 22),
 ('Momo', 22),
 ('Princess', 22),
 ('Luna', 22),
 ('Buddy', 21),
 ('Tommy', 21),
 ('Fluffy', 21),
 ('Panda', 21),
 ('Peanut', 20),
 ('Cookie', 20),
 ('Snow', 20),
 ('Bobo', 20),
 ('Toby', 19),
 ('Mickey', 18),
 ('Shadow', 18),
 ('Blacky', 18),
 ('Teddy', 17),
 ('Lulu', 17),
 ('Coffee', 17),
 ('Oyen', 17),
 ('Leo', 17),
 ('Minnie', 17),
 ('Bailey', 17),
 ('Pepper', 16),
 ('Boy', 16),
 ('Jojo', 16),
 ('Cutie', 16),
 ('Milky', 15),
 ('Mocha', 15),
 ('

## drop "RescuerID" and "Name"

In [5]:
csv_dataset["Name"].unique().shape, csv_dataset["RescuerID"].unique().shape

((9061,), (5595,))

In [6]:
csv_dataset.drop(["RescuerID","Name","Description"], axis = 1, inplace=True)

In [7]:
csv_dataset.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

## add nlp features

In [8]:
def get_max_abslute(sentences):
    max_ = 0
    real = 0
    for sentence in sentences:
        tmp = abs(sentence['sentiment']['score'])
        if tmp > max_:
            max_ = tmp
            real = sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
    return real


def get_min_abslute(sentences):
    min_ = float("inf")
    real = 0
    for sentence in sentences:
        tmp = abs(sentence['sentiment']['score'])
        if tmp < min_:
            min_ = tmp
            real = sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
    return real


def get_mean_abslute(sentences):
    sum_ = 0
    count = 0
    real = 0
    for sentence in sentences:
        sum_ += sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
        count += 1
    
    return sum_ / count
    

def func(value):
    if type(value) == type("value"):
        return 0
    else:
        return value
    
    
def get_sentiment_dict(path):
    score_max = {}
    score_min = {}
    score_mean = {}
    
    document_score = {}
    document_magnitude = {}

    
    
    for file in os.listdir(path):
        if file.endswith(".json"):
            PetID = file.split(".")[0]
            with open(path + file,"r") as f:
                content = json.loads(f.read())

            
            score_max[PetID] = get_max_abslute(content['sentences'])
            score_min[PetID] = get_min_abslute(content['sentences'])
            score_mean[PetID] = get_mean_abslute(content['sentences'])
            
            document_score[PetID] = content['documentSentiment']['score']
            document_magnitude[PetID] = content['documentSentiment']['magnitude']
    
    return score_max,score_min,score_mean,document_score,document_magnitude

In [9]:
train_nlp_path = working_dir + "/train_sentiment/"
score_max,score_min,score_mean,document_score,document_magnitude = get_sentiment_dict(train_nlp_path)


csv_dataset["score_max"] = csv_dataset.index
csv_dataset["score_max"] = csv_dataset["score_max"].replace(score_max)
csv_dataset["score_max"] = csv_dataset["score_max"].apply(func)

csv_dataset["score_min"] = csv_dataset.index
csv_dataset["score_min"] = csv_dataset["score_min"].replace(score_min)
csv_dataset["score_min"] = csv_dataset["score_min"].apply(func)


csv_dataset["score_mean"] = csv_dataset.index
csv_dataset["score_mean"] = csv_dataset["score_mean"].replace(score_mean)
csv_dataset["score_mean"] = csv_dataset["score_mean"].apply(func)


csv_dataset["document_score"] = csv_dataset.index
csv_dataset["document_score"] = csv_dataset["document_score"].replace(document_score)
csv_dataset["document_score"] = csv_dataset["document_score"].apply(func)


csv_dataset["document_magnitude"] = csv_dataset.index
csv_dataset["document_magnitude"] = csv_dataset["document_magnitude"].replace(document_magnitude)
csv_dataset["document_magnitude"] = csv_dataset["document_magnitude"].apply(func)

In [10]:
csv_dataset.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed', 'score_max', 'score_min', 'score_mean',
       'document_score', 'document_magnitude'],
      dtype='object')

### dummy category features and standard numerical features

1. category:  
    ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',  
    'Color3', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health',  
    'State','score_max', 'score_min', 'score_mean','document_score', 'document_magnitude']

2. numerical:
    ['Age','MaturitySize','FurLength', 'Quantity', 'Fee','VideoAmt', 'PhotoAmt']

In [11]:
numerical_features = ['Age','MaturitySize','FurLength', 'Quantity', 'Fee','VideoAmt', 'PhotoAmt','Breed1', 'Breed2','score_max', 'score_min', 'score_mean',
       'document_score', 'document_magnitude']
non_numerical_features = ['Type', 'Gender', 'Color1', 'Color2', 'Color3', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health','State']

scaler = StandardScaler()
csv_dataset.loc[:,numerical_features] = scaler.fit_transform(csv_dataset.loc[:,numerical_features])
csv_dataset = pd.get_dummies(csv_dataset, dummy_na=True, columns = non_numerical_features)

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [12]:
labels = csv_dataset["AdoptionSpeed"]
csv_dataset.drop(labels = ["AdoptionSpeed"], axis = 1, inplace=True)

labels.shape, csv_dataset.shape

((14993,), (14993, 75))

In [13]:
with open("labels.pkl","wb") as f:
    pkl.dump(labels,f)

with open("csv_dataset.pkl","wb") as f:
    pkl.dump(csv_dataset,f)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    csv_dataset, labels, test_size=0.2, random_state=0)

parameters = {
    'gamma': [0.1,0,2],
    'max_depth': [6,8,10,12],               
    'subsample': [0.8,0.9,1.0],              
    'colsample_bytree': [0.8,0.9,1.0],       
    'min_child_weight': [3,5],
    'eta': [0.001, 0.05, 0.1]
}

model = xgb.XGBClassifier(
        objective= 'multi:softmax',
        reg_lambda= 2,                   
        num_class = 5,
        nthread = 4,
        silent = 1,
)

model = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring="accuracy")
model.fit(X_train, y_train)
print("Best parameters set found on development set:\n")
print(model.best_params_)

KeyboardInterrupt: 

In [15]:
pred = model.predict(X_test)
print(sum(pred == y_test)/len(pred))
print(classification_report(pred,y_test))

0.41347115705235077
              precision    recall  f1-score   support

         0.0       0.08      0.60      0.14        10
         1.0       0.34      0.39      0.36       573
         2.0       0.41      0.35      0.38       935
         3.0       0.24      0.39      0.30       397
         4.0       0.64      0.49      0.56      1084

   micro avg       0.41      0.41      0.41      2999
   macro avg       0.34      0.44      0.35      2999
weighted avg       0.46      0.41      0.43      2999



In [297]:
labels_encoder = OneHotEncoder()
one_hot_labels = labels_encoder.fit_transform(labels.values.reshape((labels.shape[0],1)))
one_hot_labels.shape

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(14993, 5)

In [308]:
model = Sequential()

model.add(Dense(units=100, input_dim= X_train.shape[1], activation = "relu"))
model.add(Dropout(rate = 0.3))
model.add(Dense(units=50,activation = "relu"))
model.add(Dropout(rate = 0.5))
model.add(Dense(units=20,activation = "relu"))
model.add(Dropout(rate = 0.3))
model.add(Dense(units=5))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', 
              optimizer= "Adam",
              metrics=['accuracy'])

In [356]:
X_train, X_test, y_train, y_test = train_test_split(
    csv_dataset, one_hot_labels, test_size=0.2, random_state=0)

# class_weights = compute_class_weight('balanced',np.unique(labels),labels)
# sample_weights = np.array([class_weights[index[0,0]] for index in np.argmax(y_train,axis = 1)])
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=500, batch_size= 256)
#model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=50, batch_size= 256, sample_weight=sample_weights)

Train on 11994 samples, validate on 2999 samples
Epoch 1/500
11994/11994 [==============================] - 0s 21us/step - loss: 1.2197 - acc: 0.4588 - val_loss: 1.4457 - val_acc: 0.3791
Epoch 2/500
11994/11994 [==============================] - 0s 21us/step - loss: 1.2190 - acc: 0.4548 - val_loss: 1.4439 - val_acc: 0.3811
Epoch 3/500
11994/11994 [==============================] - 0s 20us/step - loss: 1.2130 - acc: 0.4555 - val_loss: 1.4424 - val_acc: 0.3741
Epoch 4/500
11994/11994 [==============================] - 0s 20us/step - loss: 1.2113 - acc: 0.4628 - val_loss: 1.4475 - val_acc: 0.3821
Epoch 5/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.2084 - acc: 0.4663 - val_loss: 1.4550 - val_acc: 0.3781
Epoch 6/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.2069 - acc: 0.4630 - val_loss: 1.4481 - val_acc: 0.3691
Epoch 7/500
11994/11994 [==============================] - 0s 21us/step - loss: 1.2211 - acc: 0.4541 - val_loss: 1.4417 - v

Epoch 118/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1745 - acc: 0.4837 - val_loss: 1.4726 - val_acc: 0.3768
Epoch 119/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1744 - acc: 0.4803 - val_loss: 1.4761 - val_acc: 0.3721
Epoch 120/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1716 - acc: 0.4831 - val_loss: 1.4768 - val_acc: 0.3761
Epoch 121/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1787 - acc: 0.4787 - val_loss: 1.4673 - val_acc: 0.3721
Epoch 122/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1776 - acc: 0.4783 - val_loss: 1.4702 - val_acc: 0.3825
Epoch 123/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1763 - acc: 0.4840 - val_loss: 1.4737 - val_acc: 0.3721
Epoch 124/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1781 - acc: 0.4796 - val_loss: 1.4641 - val_acc: 0.3781
Epoch 125/500
11994/

11994/11994 [==============================] - 0s 21us/step - loss: 1.1587 - acc: 0.4914 - val_loss: 1.4957 - val_acc: 0.3758
Epoch 235/500
11994/11994 [==============================] - 0s 21us/step - loss: 1.1582 - acc: 0.4866 - val_loss: 1.4885 - val_acc: 0.3768
Epoch 236/500
11994/11994 [==============================] - 0s 20us/step - loss: 1.1572 - acc: 0.4927 - val_loss: 1.4901 - val_acc: 0.3705
Epoch 237/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1624 - acc: 0.4883 - val_loss: 1.4974 - val_acc: 0.3668
Epoch 238/500
11994/11994 [==============================] - 0s 20us/step - loss: 1.1602 - acc: 0.4887 - val_loss: 1.4943 - val_acc: 0.3708
Epoch 239/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1582 - acc: 0.4867 - val_loss: 1.4884 - val_acc: 0.3748
Epoch 240/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1600 - acc: 0.4919 - val_loss: 1.4937 - val_acc: 0.3688
Epoch 241/500
11994/11994 [=======

11994/11994 [==============================] - 0s 19us/step - loss: 1.1482 - acc: 0.4973 - val_loss: 1.5118 - val_acc: 0.3755
Epoch 351/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1512 - acc: 0.4977 - val_loss: 1.5026 - val_acc: 0.3761
Epoch 352/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1498 - acc: 0.4997 - val_loss: 1.4976 - val_acc: 0.3715
Epoch 353/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1514 - acc: 0.4956 - val_loss: 1.5025 - val_acc: 0.3738
Epoch 354/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1478 - acc: 0.4926 - val_loss: 1.5025 - val_acc: 0.3728
Epoch 355/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1480 - acc: 0.4972 - val_loss: 1.5009 - val_acc: 0.3778
Epoch 356/500
11994/11994 [==============================] - 0s 18us/step - loss: 1.1457 - acc: 0.4996 - val_loss: 1.5021 - val_acc: 0.3705
Epoch 357/500
11994/11994 [=======

11994/11994 [==============================] - 0s 19us/step - loss: 1.1423 - acc: 0.4997 - val_loss: 1.5157 - val_acc: 0.3775
Epoch 467/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1402 - acc: 0.5021 - val_loss: 1.5113 - val_acc: 0.3755
Epoch 468/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1495 - acc: 0.4985 - val_loss: 1.5131 - val_acc: 0.3771
Epoch 469/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1373 - acc: 0.5038 - val_loss: 1.5129 - val_acc: 0.3708
Epoch 470/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1306 - acc: 0.5043 - val_loss: 1.5195 - val_acc: 0.3675
Epoch 471/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1400 - acc: 0.4994 - val_loss: 1.5147 - val_acc: 0.3715
Epoch 472/500
11994/11994 [==============================] - 0s 19us/step - loss: 1.1348 - acc: 0.5051 - val_loss: 1.5217 - val_acc: 0.3781
Epoch 473/500
11994/11994 [=======